In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 研究区
roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','武汉市'))
# roi = ee.Geometry.Rectangle([89.8565, 34.6503,91.4664, 35.8282])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 图像可视化参数
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

## 计算相关水体指数，包括NDVI、mNDWI、AWEI
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)

# 计算研究区域内，影像面积
def ImageArea(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.set({'imagearea': imageareas.get('B2')})

In [5]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi','AWEI']
# 分类标签
label = 'waterclass'

# 加载图像

In [6]:
image_id = 'LC08_123039_20180915'
id = 'LANDSAT/LC08/C01/T1_SR/' + image_id
image2 = water_index(ee.Image(id))

Map.addLayer(image2,visParams,'image2')
Map.addLayer(maskL8sr(image2),visParams,'mask image2')

# 不同的样本比例

## 陆地：水体 ≈ 7：3

In [25]:
## 导入采集的样本集
samples_73 = ee.FeatureCollection('users/311605001111/wuhan_8_3')
print(samples_73.first().getInfo())
print('total sample number:{}'.format(samples_73.size().getInfo()))
a = samples_73.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())
a = samples_73.filter(ee.Filter.eq('waterclass',0))
print('陆地样本的数目：',a.size().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.46774599337513, 30.65465203864643]}, 'id': '0000000000000000033e', 'properties': {'AWEI': -2612.5, 'B2': 748, 'B3': 1018, 'B4': 1062, 'B5': 2036, 'B6': 1692, 'B7': 1254, 'Image_id': 'LC08_122039_20180417', 'NDVI': 0.314396384764364, 'mNDWI': -0.248708487084871, 'ndvi_mndwi': 0.563104871849235, 'waterclass': 0}}
total sample number:1135
水体样本的数目： 310
陆地样本的数目： 825


In [26]:
trainedClassifier_73 = ee.Classifier.smileRandomForest(20).train(samples_73,label,bands)
image_73 = maskL8sr(image2).select(bands).classify(trainedClassifier_73).eq(1).rename('waterclass')
Map.addLayer(image_73.selfMask(),{'palette':['blue']},'image_73')

## 陆地：水体 ≈ 6：4

In [23]:
## 导入采集的样本集
samples_64 = ee.FeatureCollection('users/311605001111/wuhan_8_5')
print(samples_64.first().getInfo())
print('total sample number:{}'.format(samples_64.size().getInfo()))
a = samples_64.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())
a = samples_64.filter(ee.Filter.eq('waterclass',0))
print('陆地样本的数目：',a.size().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.26044238540042, 30.673799418569924]}, 'id': '00000000000000000343', 'properties': {'AWEI': -208.25, 'B2': 746, 'B3': 957, 'B4': 949, 'B5': 1148, 'B6': 949, 'B7': 805, 'Image_id': 'LC08_123039_20181017', 'NDVI': 0.094897472579876, 'mNDWI': 0.004197271773347, 'ndvi_mndwi': 0.090700200806529, 'waterclass': 0}}
total sample number:1340
水体样本的数目： 515
陆地样本的数目： 825


In [24]:
trainedClassifier_64 = ee.Classifier.smileRandomForest(20).train(samples_64,label,bands)
image_64 = maskL8sr(image2).select(bands).classify(trainedClassifier_64).eq(1).rename('waterclass')
Map.addLayer(image_64.selfMask(),{'palette':['green']},'image_64')

## 陆地：水体 ≈ 5：5

In [21]:
## 导入采集的样本集
samples_55 = ee.FeatureCollection('users/311605001111/wuhan_8_8')
print(samples_55.first().getInfo())
print('total sample number:{}'.format(samples_55.size().getInfo()))
a = samples_55.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())
a = samples_55.filter(ee.Filter.eq('waterclass',0))
print('陆地样本的数目：',a.size().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.46774599337513, 30.65465203864643]}, 'id': '000000000000000004e8', 'properties': {'AWEI': -2612.5, 'B2': 748, 'B3': 1018, 'B4': 1062, 'B5': 2036, 'B6': 1692, 'B7': 1254, 'Image_id': 'LC08_122039_20180417', 'NDVI': 0.314396384764364, 'mNDWI': -0.248708487084871, 'ndvi_mndwi': 0.563104871849235, 'waterclass': 0}}
total sample number:1673
水体样本的数目： 848
陆地样本的数目： 825


In [22]:
trainedClassifier_55 = ee.Classifier.smileRandomForest(20).train(samples_55,label,bands)
image_55 = maskL8sr(image2).select(bands).classify(trainedClassifier_55).eq(1).rename('waterclass')
Map.addLayer(image_55.selfMask(),{'palette':['red']},'image_55')

## 陆地：水体 ≈ 4：6

In [19]:
## 导入采集的样本集
samples_46 = ee.FeatureCollection('users/311605001111/wuhan_8_11')
print(samples_46.first().getInfo())
print('total sample number:{}'.format(samples_46.size().getInfo()))
a = samples_46.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())
a = samples_46.filter(ee.Filter.eq('waterclass',0))
print('陆地样本的数目：',a.size().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.73338356133098, 30.693209885455573]}, 'id': '00000000000000000661', 'properties': {'AWEI': -5.5, 'B2': 709, 'B3': 851, 'B4': 775, 'B5': 1020, 'B6': 786, 'B7': 532, 'Image_id': 'LC08_122039_20181127', 'NDVI': 0.136490250696379, 'mNDWI': 0.039706780696396, 'ndvi_mndwi': 0.096783469999983, 'waterclass': 0}}
total sample number:1950
水体样本的数目： 1125
陆地样本的数目： 825


In [20]:
trainedClassifier_46 = ee.Classifier.smileRandomForest(20).train(samples_46,label,bands)
image_46 = maskL8sr(image2).select(bands).classify(trainedClassifier_46).eq(1).rename('waterclass')
Map.addLayer(image_46.selfMask(),{'palette':['yellow']},'image_46')

## 陆地：水体 ≈ 3：7

In [15]:
## 导入采集的样本集
samples_37 = ee.FeatureCollection('users/311605001111/wuhan_8_15')
print(samples_37.first().getInfo())
print('total sample number:{}'.format(samples_37.size().getInfo()))
a = samples_37.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())
a = samples_37.filter(ee.Filter.eq('waterclass',0))
print('陆地样本的数目：',a.size().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.36979737963514, 30.374312814517335]}, 'id': '00000000000000000660', 'properties': {'AWEI': -2435, 'B2': 846, 'B3': 1214, 'B4': 1327, 'B5': 1980, 'B6': 1942, 'B7': 1732, 'Image_id': 'LC08_123039_20180611', 'NDVI': 0.197459933474448, 'mNDWI': -0.230671736375158, 'ndvi_mndwi': 0.428131669849607, 'waterclass': 0}}
total sample number:2361
水体样本的数目： 1536
陆地样本的数目： 825


In [16]:
trainedClassifier_37 = ee.Classifier.smileRandomForest(20).train(samples_37,label,bands)
image_37 = maskL8sr(image2).select(bands).classify(trainedClassifier_37).eq(1).rename('waterclass')
Map.addLayer(image_37.selfMask(),{'palette':['violet']},'image_37')